# Prep sequences and metadata for `augur`

In [ ]:
import Bio.SeqIO

import pandas as pd

In [ ]:
# Get variables from `snakemake`
seq_files = snakemake.input.seqs
accession_info_csv = snakemake.input.accession_info
output_sequences = snakemake.output.sequences
output_metadata = snakemake.output.metadata

In [ ]:
accession_info = pd.read_csv(accession_info_csv)
accessions = set(accession_info["accession"])

seqs = []
accessions_found = set()
for seq_file in seq_files:
    iseqs = list(Bio.SeqIO.parse(seq_file, "fasta"))
    print(f"Read {len(iseqs)} sequences from {seq_file}")
    for seq in iseqs:
        if not (29000 <= len(seq) <= 31000):
            raise ValueError(f"{seq=} has invalid length {len(seq)}")
        if seq.id not in accessions:
            seq.id = seq.id.split(".")[0]  # remove version from Genbank sequences
        assert seq.id in accessions, f"{seq.id=} not in accessions"
        seqs.append(seq)
        if seq.id in accessions_found:
            raise ValueError(f"duplicate sequences for {seq.id}")
        accessions_found.add(seq.id)
        
print(f"Overall processed {len(seqs)} sequences for the {len(accessions)} accessions")

print(f"Writing the sequences to {output_sequences}")
Bio.SeqIO.write(seqs, output_sequences, "fasta")

print(f"Writing the metadata to {output_metadata}")
accession_info.query("accession in @accessions_found").to_csv(output_metadata, index=False, sep="\t")